In [7]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import fetch_olivetti_faces
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

class TorchNN(nn.Module):
    def __init__(self):  
        super().__init__()

        self.linear1 = nn.Linear(4096, 512)  # 输入改为 4096
        self.bn1 = nn.BatchNorm1d(512)
        self.linear2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.linear3 = nn.Linear(512, 40)  # Olivetti Faces 有 40 个类别

        self.drop = nn.Dropout(p=0.3)
        self.act = nn.ReLU()

    def forward(self, input_tensor):
        out = self.linear1(input_tensor)
        out = self.bn1(out)
        out = self.act(out)
        out = self.drop(out)
        out = self.linear2(out)
        out = self.bn2(out)
        out = self.act(out)
        out = self.drop(out)
        final = self.linear3(out)

        return final

# 设备检测
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#  加载 Olivetti Faces 数据集
data = fetch_olivetti_faces(shuffle=True)
X, y = data.images, data.target  # X.shape = (400, 64, 64), y.shape = (400,)

#  数据预处理（归一化 & 转换）
scaler = StandardScaler()
X = scaler.fit_transform(X.reshape(len(X), -1))  # (400, 4096)

# 转换为 PyTorch 张量
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

#  创建 DataLoader（减少 BATCH_SIZE 以防内存溢出）
BATCH_SIZE = 16  
dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


# 训练不同优化器并对比 Loss
EPOCHS = 100
loss_fn = nn.CrossEntropyLoss()

optimizers = {
    "SGD": optim.SGD,
    "Adam": optim.Adam,
    "RMSprop": optim.RMSprop
}

results = {}

for opt_name, opt_class in optimizers.items():
    print(f"\nTraining with {opt_name} optimizer...")
    model = TorchNN().to(device)  # 发送至 GPU
    optimizer = opt_class(model.parameters(), lr=0.001)

    for epoch in range(EPOCHS):
        total_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            optimizer.zero_grad()
            output = model(batch_X)
            loss = loss_fn(output, batch_y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}")

    results[opt_name] = avg_loss

# 输出最终 Loss 结果
print("\nFinal Loss Comparison:")
for opt_name, loss in results.items():
    print(f"{opt_name}: {loss:.4f}")


Using device: cuda

Training with SGD optimizer...
Epoch 1/100, Loss: 3.7204
Epoch 2/100, Loss: 3.4838
Epoch 3/100, Loss: 3.2458
Epoch 4/100, Loss: 3.0208
Epoch 5/100, Loss: 2.9030
Epoch 6/100, Loss: 2.8162
Epoch 7/100, Loss: 2.6547
Epoch 8/100, Loss: 2.5118
Epoch 9/100, Loss: 2.4519
Epoch 10/100, Loss: 2.3327
Epoch 11/100, Loss: 2.2483
Epoch 12/100, Loss: 2.2029
Epoch 13/100, Loss: 2.0843
Epoch 14/100, Loss: 2.0237
Epoch 15/100, Loss: 1.9924
Epoch 16/100, Loss: 1.9107
Epoch 17/100, Loss: 1.8328
Epoch 18/100, Loss: 1.8260
Epoch 19/100, Loss: 1.7013
Epoch 20/100, Loss: 1.6427
Epoch 21/100, Loss: 1.6053
Epoch 22/100, Loss: 1.5767
Epoch 23/100, Loss: 1.5330
Epoch 24/100, Loss: 1.4766
Epoch 25/100, Loss: 1.4283
Epoch 26/100, Loss: 1.3606
Epoch 27/100, Loss: 1.3489
Epoch 28/100, Loss: 1.2939
Epoch 29/100, Loss: 1.2666
Epoch 30/100, Loss: 1.2201
Epoch 31/100, Loss: 1.2130
Epoch 32/100, Loss: 1.1687
Epoch 33/100, Loss: 1.1175
Epoch 34/100, Loss: 1.0990
Epoch 35/100, Loss: 1.0966
Epoch 36/100,